# 动态图

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/zh_cn/model_train/program_form/mindspore_pynative.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/zh_cn/model_train/program_form/mindspore_pynative.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.svg)](https://gitee.com/mindspore/docs/blob/master/docs/mindspore/source_zh_cn/model_train/program_form/pynative.ipynb)

## 基础能力

在MindSpore中，动态图模式又被称为PyNative模式，为默认模式，也可以通过`set_context(mode=PYNATIVE_MODE)`来设置成动态图模式。在脚本开发和网络流程调试中，动态图模式下更容易调试，支持执行单算子、普通函数和网络、以及单独求梯度的操作。

在PyNative模式下，用户可以使用完整的Python API，此外针对使用MindSpore提供的API时，框架会根据用户选择的硬件平台（Ascend，GPU，CPU），将算子API的操作在对应的硬件平台上执行，并返回相应的结果。框架整体的执行过程如下：

![process](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/docs/mindspore/source_zh_cn/design/images/framework.png)

通过前端的Python API，调用到框架层，最终到相应的硬件设备上进行计算。例如：完成一个加法。

In [2]:
import numpy as np
import mindspore as ms
import mindspore.ops as ops

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
x = ms.Tensor(np.ones([1, 3, 3, 4]).astype(np.float32))
y = ms.Tensor(np.ones([1, 3, 3, 4]).astype(np.float32))
output = ops.add(x, y)
print(output.asnumpy())

[[[[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]

  [[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]

  [[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]]]


此例中，当调用到Python接口ops.add(x, y)时，会将Python的接口调用通过Pybind11调用到框架的C++层，转换成C++的调用，接着框架会根据用户设置的device_target选择对应的硬件设备，在该硬件设备上执行add这个操作。

从上述原理可以看到，在PyNative模式下，Python脚本代码会根据Python的语法进行执行，而执行过程中涉及到MindSpore的API，会根据用户设置在不同的硬件上进行执行，从而进行加速。因此，在PyNative模式下，用户可以随意使用Python的语法以及调试方法。例如可以使用常见的PyCharm、VS Code等IDE进行代码的调试。

## 动静结合

### JIT

#### 概述

#### PIJit

MindSpore提供了一种在代码无需修改的情况下，直接将用户的动态图代码转换成静态图的功能——PIJit。该功能同时兼顾性能和易用性，去除动静模式切换的代价，真正做到动静统一。它基于Python字节码的分析，对Python的执行流进行图捕获，可以以静态图方式运行的子图便以静态图方式运行，Python语法不支持的子图便以动态图方式运行，同时通过修改调整字节码的方式链接静态图，达到动静混合执行。在满足易用性的前提下，尽可能地提高性能。

##### PIJit包含以下功能

- 1. 图捕获：对字节码预处理，动态跟踪解释执行，识别MindSpore可入图操作，提供裂图功能保证函数（字节码）功能的正确性。
- 2. 字节码支持：当前支持Python3.7、Python3.8、Python3.9和Python3.10的字节码。
- 3. 图优化：对图中生成的字节码进行优化，包括分支裁剪、字节码筛选、函数字节码内联、常量折叠等功能。
- 4. 异常捕获机制：支持with、try-except语法。
- 5. 支持循环处理：通过模拟字节码的操作栈实现图捕获、裂图等特性。
- 6. UD分析：通过变量的user-def链分析的方法，解决部分参数类型不能作为静态图的返回值问题（函数、Bool、None），同时减少无用的入参，提高图的执行效率，减少数据的拷贝。
- 7. 副作用分析处理：弥补静态图的副作用处理上的劣势，根据不同场景，收集记录产生副作用的变量及字节码，在保证程序语义的基础上，在静态图外补充副作用的处理。
- 8. 守护门禁：门禁（Guard）记录了子图/优化进入的输入需要满足的条件，检查输入是否适合对应的子图优化。
- 9. Cache：图管理（Cache）则缓存了子图/优化和门禁（Guard）对应关系。
- 10. Dynamic Shape和Symbolic Shape: 使用input_signature支持Tensor/Tensor List/Tensor Tuple的DynamicShape和SymblicShape作为输入提示。同时支持自动多次运行后，识别Dynamic Shape。
- 11. 即时跟踪编译: 在跟踪和字节码分析过程中，支持算子等类型推导。
- 12. 自动混合精度: 支持原生mindspore.nn.Cell的自动混合精度能力。

##### 使用方式

def jit(fn=None, input_signature=None, hash_args=None, jit_config=None, mode="PIJit"):

原Jit功能使用mode="PSJit"，新特性PIJit使用mode="PIJit"，jit_config传递一个参数字典，可以提供一些优化和调试选项。如：print_after_all可以打印入图的字节码和裂图信息，loop_unrolling可以提供循环展开功能，enable_dynamic_shape使能动态Shape。

##### 使用限制

- 不支持在静态图模式下，运行带装饰@jit(mode=\"PIJit\")的函数，此时该装饰@jit(mode=\"PIJit\")视为无效。
- 不支持在@jit(mode=\"PSJit\")装饰的函数内部调用带装饰@jit(mode=\"PIJit\")的函数，该装饰 @jit(mode=\"PIJit\")视为无效。

#### PSJit

### Shard

